# Extração de Features - 5 Braços Experimentais

Este notebook extrai features para os 5 braços experimentais:

1. **Baseline CNN:** ResNet/U-Net padrão
2. **ViT Puro:** ViT pré-treinado no ImageNet
3. **ViT + Contrastive:** Usando domain_specific_cl
4. **ViT + MIM:** Usando MIM-Med3D
5. **ViT + Sparse:** Aplicação de esparsidade nas features

## Estrutura de Saída

As features serão salvas em:
```
features/
├── baseline_cnn/
├── vit_pure/
├── vit_contrastive/
├── vit_mim/
└── vit_sparse/
```


In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from transformers import TFAutoModel, AutoFeatureExtractor

# Configuração de GPU
from tensorflow.keras.mixed_precision import set_global_policy
if len(tf.config.list_physical_devices('GPU')) > 0:
    policy = tf.keras.mixed_precision.Policy('mixed_float16')
    set_global_policy(policy)
    print("GPU ativa - Mixed Precision habilitado")
else:
    print("GPU não disponível - Usando CPU")

# Configuração de diretórios
BASE_DIR = Path("../")
DATA_DIR = BASE_DIR / "data" / "processed"
FEATURES_DIR = BASE_DIR / "features"
FEATURES_DIR.mkdir(exist_ok=True)

# Criar diretórios para cada braço experimental
EXPERIMENTAL_ARMS = [
    "baseline_cnn",
    "vit_pure",
    "vit_contrastive",
    "vit_mim",
    "vit_sparse"
]

for arm in EXPERIMENTAL_ARMS:
    (FEATURES_DIR / arm).mkdir(exist_ok=True)

print(f"Diretórios criados em: {FEATURES_DIR.absolute()}")
